In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Conv1D, GlobalAvgPool2D, GlobalAveragePooling1D, \
    Dropout, Dense, Activation, Concatenate, Multiply, MaxPool2D, Add,  \
    LSTM, Bidirectional, Conv2D, AveragePooling2D, BatchNormalization, Flatten, GlobalAveragePooling2D, \
    GlobalMaxPooling2D, Reshape, Permute, multiply, Lambda, add, subtract, MaxPooling2D, GRU, ReLU, MaxPooling1D 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score, roc_curve
from keras.models import Model, load_model
from keras import backend as K
from sklearn.model_selection import KFold
from keras.layers import Embedding
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.regularizers import l1, l2
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from Bio import SeqIO
mpl.use('TkAgg')
import warnings
warnings.filterwarnings("ignore")
import math
from tensorflow.keras import layers
from tensorflow import keras
from keras import initializers

2024-06-12 12:12:30.561560: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 12:12:30.597002: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 12:12:31.177004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # 不全部占满显存，按需分配当allow_growth设置为True时，分配器将不会指定所有的GPU内存，而是根据需求增长
session = tf.compat.v1.Session(config=config)

2024-06-12 12:12:40.121715: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9720 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


In [3]:
def read_fasta(file_path):
    '''File_path: Path to the fasta file
       Returns: List of sequence
    '''
    one=list(SeqIO.parse(file_path,'fasta'))
    return one

In [4]:
def protein_to_Kmer(seqs,K):
    numeric_sequences = []
    base_to_index = {'A': 0, 'C': 1, 'D': 2, 'E': 3,
                     'F': 4, 'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9, 'M': 10,
                     'N': 11, 'P': 12, 'Q': 13, 'R': 14, 'S': 15, 'T': 16, 
                     'V': 17, 'W': 18, 'Y': 19,'X': 20
                     }
    for dna_sequence in seqs:
        numeric_sequence = []
        for base in dna_sequence:
            numeric_sequence.append(base_to_index[base])

        kmer_sequence = []
        for i in range(len(numeric_sequence) - (K - 1)):
            kmer = numeric_sequence[i:i + K]
            kmer_marge = [int("".join(map(str, kmer)))]
            kmer_sequence.append(kmer_marge)

        numeric_sequences.append(kmer_sequence)
    return numeric_sequences

In [5]:
# Read the training set
    
train_pos_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_pos_15_train.txt'))
train_neg_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_neg_15_train.txt'))

train_seqs = np.concatenate( (train_pos_seqs, train_neg_seqs), axis=0 )

train = np.array(protein_to_Kmer(train_seqs,1)).astype(np.float32)

train_label = np.array( [1] * 2866 + [0] * 2866 ).astype( np.float32 )
train_label = to_categorical( train_label, num_classes=2 )

# Read the testing set
test_pos_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_pos_15_test.txt'))
test_neg_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_neg_15_test.txt'))

test_seqs = np.concatenate((test_pos_seqs, test_neg_seqs), axis=0)

test = np.array(protein_to_Kmer(test_seqs,1)).astype(np.float32)

test_label = np.array([1] * 318 + [0] * 318).astype(np.float32)
test_label = to_categorical(test_label, num_classes=2)

In [6]:
# Performance evaluation
def show_performance(y_true, y_pred):

    TP, FP, FN, TN = 0, 0, 0, 0

    for i in range(len(y_true)):
        if y_true[i] == 1:
            if y_pred[i] > 0.5:
                TP += 1
            else:
                FN += 1
        if y_true[i] == 0:
            if y_pred[i] > 0.5:
                FP += 1
            else:
                TN += 1


    Sn = TP / (TP + FN + 1e-06)

    Sp = TN / (FP + TN + 1e-06)

    Acc = (TP + TN) / len(y_true)

    MCC = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) + 1e-06)

    return Sn, Sp, Acc, MCC

def performance_mean(performance):
    print('Sn = %.4f ± %.4f' % (np.mean(performance[:, 0]), np.std(performance[:, 0])))
    print('Sp = %.4f ± %.4f' % (np.mean(performance[:, 1]), np.std(performance[:, 1])))
    print('Acc = %.4f ± %.4f' % (np.mean(performance[:, 2]), np.std(performance[:, 2])))
    print('Mcc = %.4f ± %.4f' % (np.mean(performance[:, 3]), np.std(performance[:, 3])))
    print('Auc = %.4f ± %.4f' % (np.mean(performance[:, 4]), np.std(performance[:, 4])))

In [7]:
# Building the model
def build_model(windows=15,embed_dim=64, weight_decay=1e-4,num_heads=8,ff_dim=64):

    print('*' * 30 + 'LSTM' + '*' * 30)
    input_1 = Input(shape=(windows,))
    # Word embedding coding
    embedding = Embedding(21, embed_dim, input_length=15)
    x_1 = embedding(input_1)
    print(x_1.shape)
    #transformer = TransformerBlock(embed_dim, num_heads, ff_dim)(x_1)
    #print(transformer.shape)

    x_1 = LSTM(32,return_sequences=True,activation = 'relu')(x_1)
    x_1 = Dropout(0.5)(x_1)
    x_1 = LSTM(16,return_sequences=True,activation = 'relu')(x_1)
    x_1 = Dropout(0.5)(x_1)
    print(x_1.shape)
    
    x = Flatten()(x_1)
    x = Dropout(0.5)(x)
    x = Dense(units=2, activation='softmax')(x)


    inputs = [input_1]
    outputs = [x]

    model = Model(inputs=inputs, outputs=outputs, name="Kbhb")

    optimizer = Adam(learning_rate=1e-3, epsilon=1e-7)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [8]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('./models_duibi_LSTM/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./models_duibi_LSTM/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./models_duibi_LSTM/model_test.h5')

del model

model = load_model('./models_duibi_LSTM/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************LSTM******************************
(None, 15, 64)


2024-06-03 08:12:02.876190: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8029 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 16)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-03 08:12:07.148431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-03 08:12:07.239042: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f116400e100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-03 08:12:07.239077: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-03 08:12:07.246244: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-03 08:12:07.428534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
2024-06-03 08:12:07.472295: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. 

  1/162 [..............................] - ETA: 13:06 - loss: 0.6926 - accuracy: 0.5312

...

162/162 [==============================] - 10s 63ms/step - loss: 0.5216 - accuracy: 0.7414 - val_loss: 0.5342 - val_accuracy: 0.7422
Epoch 19/200
162/162 [==============================] - 10s 63ms/step - loss: 0.5205 - accuracy: 0.7462 - val_loss: 0.5412 - val_accuracy: 0.7456
Epoch 20/200
162/162 [==============================] - 10s 62ms/step - loss: 0.5149 - accuracy: 0.7451 - val_loss: 0.5363 - val_accuracy: 0.7387
Epoch 21/200
162/162 [==============================] - 10s 62ms/step - loss: 0.5048 - accuracy: 0.7507 - val_loss: 0.5274 - val_accuracy: 0.7456
Epoch 22/200
162/162 [==============================] - 10s 63ms/step - loss: 0.5093 - accuracy: 0.7513 - val_loss: 0.5242 - val_accuracy: 0.7561
Epoch 23/200
162/162 [==============================] - 10s 63ms/step - loss: 0.5007 - accuracy: 0.7606 - val_loss: 0.5800 - val_accuracy: 0.7021
Epoch 24/200
162/162 [==============================] - 10s 63ms/step - loss: 0.4932 - accuracy: 0.7582 - val_loss: 0.5310 - val_accuracy

2024-06-03 08:17:54.200384: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 10s 63ms/step - loss: 0.4719 - accuracy: 0.7732 - val_loss: 0.4933 - val_accuracy: 0.7718
Epoch 35/200
162/162 [==============================] - 10s 64ms/step - loss: 0.4654 - accuracy: 0.7767 - val_loss: 0.5041 - val_accuracy: 0.7840
Epoch 36/200
162/162 [==============================] - 10s 63ms/step - loss: 0.4653 - accuracy: 0.7769 - val_loss: 0.5281 - val_accuracy: 0.7544
Epoch 37/200
162/162 [==============================] - 10s 63ms/step - loss: 0.4723 - accuracy: 0.7745 - val_loss: 0.5285 - val_accuracy: 0.7578
Epoch 38/200
162/162 [==============================] - 10s 63ms/step - loss: 0.4638 - accuracy: 0.7825 - val_loss: 0.5072 - val_accuracy: 0.7596
Epoch 39/200
162/162 [==============================] - 10s 63ms/step - loss: 0.4631 - accuracy: 0.7850 - val_loss: 0.5171 - val_accuracy: 0.7700
Epoch 40/200
162/162 [==============================] - 10s 62ms/step - loss: 0.4577 - accuracy: 0.7881 - val_loss: 0.5274 - val_accuracy

2024-06-03 08:43:40.543559: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 11s 63ms/step - loss: 0.1349 - accuracy: 0.9435 - val_loss: 2.7693 - val_accuracy: 0.7186
Epoch 3/200
180/180 [==============================] - 11s 63ms/step - loss: 0.1606 - accuracy: 0.9355 - val_loss: 1.6016 - val_accuracy: 0.7264
Epoch 4/200
180/180 [==============================] - 11s 64ms/step - loss: 0.1913 - accuracy: 0.9213 - val_loss: 1.7087 - val_accuracy: 0.7154
Epoch 5/200
180/180 [==============================] - 11s 62ms/step - loss: 0.1681 - accuracy: 0.9325 - val_loss: 1.2957 - val_accuracy: 0.7296
Epoch 6/200
180/180 [==============================] - 11s 63ms/step - loss: 0.1711 - accuracy: 0.9318 - val_loss: 1.7751 - val_accuracy: 0.7154
Epoch 7/200
180/180 [==============================] - 11s 63ms/step - loss: 0.1662 - accuracy: 0.9374 - val_loss: 1.8984 - val_accuracy: 0.7343
Epoch 8/200
180/180 [==============================] - 11s 62ms/step - loss: 0.1191 - accuracy: 0.9512 - val_loss: 2.6559 - val_accuracy: 0.73

In [8]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('./models_duibi_LSTM/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./models_duibi_LSTM/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./models_duibi_LSTM/model_test.h5')

del model

model = load_model('./models_duibi_LSTM/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************LSTM******************************
(None, 15, 64)


2024-06-12 12:13:10.690808: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9720 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 16)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-12 12:13:13.967083: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-12 12:13:14.017030: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb7d800e220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-12 12:13:14.017077: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-12 12:13:14.027014: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-12 12:13:14.194068: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
2024-06-12 12:13:14.237798: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. 

  1/162 [..............................] - ETA: 10:14 - loss: 0.6934 - accuracy: 0.4688

...

162/162 [==============================] - 10s 60ms/step - loss: 0.5468 - accuracy: 0.7173 - val_loss: 0.5535 - val_accuracy: 0.7282
Epoch 19/200
162/162 [==============================] - 10s 59ms/step - loss: 0.5528 - accuracy: 0.7140 - val_loss: 0.5570 - val_accuracy: 0.7178
Epoch 20/200
162/162 [==============================] - 9s 58ms/step - loss: 0.5382 - accuracy: 0.7228 - val_loss: 0.5382 - val_accuracy: 0.7317
Epoch 21/200
162/162 [==============================] - 10s 59ms/step - loss: 0.5361 - accuracy: 0.7309 - val_loss: 0.5431 - val_accuracy: 0.7352
Epoch 22/200
162/162 [==============================] - 9s 59ms/step - loss: 0.5258 - accuracy: 0.7435 - val_loss: 0.5234 - val_accuracy: 0.7300
Epoch 23/200
162/162 [==============================] - 10s 59ms/step - loss: 0.5231 - accuracy: 0.7387 - val_loss: 0.5441 - val_accuracy: 0.7334
Epoch 24/200
162/162 [==============================] - 9s 59ms/step - loss: 0.5099 - accuracy: 0.7431 - val_loss: 0.5432 - val_accuracy: 0

2024-06-12 12:18:36.270407: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 10s 60ms/step - loss: 0.4801 - accuracy: 0.7693 - val_loss: 0.5055 - val_accuracy: 0.7474
Epoch 35/200
162/162 [==============================] - 10s 59ms/step - loss: 0.4717 - accuracy: 0.7724 - val_loss: 0.5246 - val_accuracy: 0.7544
Epoch 36/200
162/162 [==============================] - 10s 59ms/step - loss: 0.4720 - accuracy: 0.7747 - val_loss: 0.5276 - val_accuracy: 0.7456
Epoch 37/200
162/162 [==============================] - 10s 59ms/step - loss: 0.4611 - accuracy: 0.7807 - val_loss: 0.5640 - val_accuracy: 0.7491
Epoch 38/200
162/162 [==============================] - 10s 59ms/step - loss: 0.4680 - accuracy: 0.7813 - val_loss: 0.5261 - val_accuracy: 0.7578
Epoch 39/200
162/162 [==============================] - 10s 59ms/step - loss: 0.4683 - accuracy: 0.7776 - val_loss: 0.5138 - val_accuracy: 0.7474
Epoch 40/200
162/162 [==============================] - 10s 59ms/step - loss: 0.4523 - accuracy: 0.7912 - val_loss: 0.5277 - val_accuracy

2024-06-12 12:42:58.751060: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 11s 59ms/step - loss: 0.1392 - accuracy: 0.9466 - val_loss: 1.8254 - val_accuracy: 0.7500
Epoch 3/200
180/180 [==============================] - 11s 61ms/step - loss: 0.1275 - accuracy: 0.9501 - val_loss: 2.4793 - val_accuracy: 0.7327
Epoch 4/200
180/180 [==============================] - 11s 60ms/step - loss: 0.1772 - accuracy: 0.9355 - val_loss: 1.7154 - val_accuracy: 0.7343
Epoch 5/200
180/180 [==============================] - 11s 59ms/step - loss: 0.1917 - accuracy: 0.9355 - val_loss: 1.6876 - val_accuracy: 0.7311
Epoch 6/200
180/180 [==============================] - 11s 59ms/step - loss: 0.1442 - accuracy: 0.9473 - val_loss: 1.6856 - val_accuracy: 0.7437
Epoch 7/200
180/180 [==============================] - 11s 60ms/step - loss: 0.1314 - accuracy: 0.9501 - val_loss: 1.4977 - val_accuracy: 0.7390
Epoch 8/200
180/180 [==============================] - 10s 58ms/step - loss: 0.1428 - accuracy: 0.9450 - val_loss: 2.1295 - val_accuracy: 0.74

In [9]:
df = pd.DataFrame({
    'true_label': test_label[:,1],
    'predicted_probability': test_score[:, 1]  # 提取正类的预测概率
})
df.to_csv('./models_duibi_LSTM/predictions_with_probabilities_lstm.csv', index=False)